In [1]:
import numpy as np
import os
from pandas import read_csv as read

In [2]:
path = "housesUSAWithCoor.csv"
data = read(path, delimiter=",")
data

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,lat,lon
0,2014-05-02 00:00:00,3.130000e+05,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA,-122.338866,47.765807
1,2014-05-02 00:00:00,2.384000e+06,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA,-122.366902,47.634540
2,2014-05-02 00:00:00,3.420000e+05,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA,-122.150440,47.367610
3,2014-05-02 00:00:00,4.200000e+05,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA,-122.113830,47.618714
4,2014-05-02 00:00:00,5.500000e+05,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA,-122.113946,47.683043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,2014-07-09 00:00:00,3.081667e+05,3.0,1.75,1510,6360,1.0,0,0,4,1510,0,1954,1979,501 N 143rd St,Seattle,WA 98133,USA,-122.352880,47.732001
4596,2014-07-09 00:00:00,5.343333e+05,3.0,2.50,1460,7573,2.0,0,0,3,1460,0,1983,2009,14855 SE 10th Pl,Bellevue,WA 98007,USA,-122.141062,47.601178
4597,2014-07-09 00:00:00,4.169042e+05,3.0,2.50,3010,7014,2.0,0,0,3,3010,0,2009,0,759 Ilwaco Pl NE,Renton,WA 98059,USA,-122.149341,47.494981
4598,2014-07-10 00:00:00,2.034000e+05,4.0,2.00,2090,6630,1.0,0,0,3,1070,1020,1974,0,5148 S Creston St,Seattle,WA 98178,USA,-122.268170,47.507746


# Новый раздел

In [3]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 27 kB/s 


In [24]:
data2 = data.drop(data.columns[14:18], axis=1)

from catboost import CatBoostRegressor
#categories = ["bedrooms",'bathrooms','sqft_living','sqft_lot',
#'floors','waterfront','view','condition','sqft_above','sqft_basement',
#'yr_built','yr_renovated','lat','lon']

X = data2.values[::, 2:16]
y = data2.values[::, 1:2]

from sklearn.model_selection import train_test_split as train
X_train, X_test, y_train, y_test = train(X, y, test_size=0.6)

clf = CatBoostRegressor(iterations=3500,learning_rate=0.03,od_type='Iter',od_wait=1500,
                             depth=6,random_strength=1,
                            l2_leaf_reg=10,
                             sampling_frequency='PerTree',
                            )
res = clf.fit(X_train, y_train,
             eval_set=(X_test,y_test),
             use_best_model=True,
             verbose=True)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0:	learn: 396001.8825622	test: 647723.1693275	best: 647723.1693275 (0)	total: 3.49ms	remaining: 12.2s
1:	learn: 390813.1899058	test: 645558.7980630	best: 645558.7980630 (1)	total: 7.63ms	remaining: 13.3s
2:	learn: 385916.5330330	test: 643500.4696980	best: 643500.4696980 (2)	total: 10.1ms	remaining: 11.8s
3:	learn: 380927.1607986	test: 641450.7763241	best: 641450.7763241 (3)	total: 12.5ms	remaining: 10.9s
4:	learn: 375715.6636276	test: 639338.6943395	best: 639338.6943395 (4)	total: 14.8ms	remaining: 10.3s
5:	learn: 371299.2352236	test: 637441.8803492	best: 637441.8803492 (5)	total: 17.2ms	remaining: 10s
6:	learn: 367468.4155197	test: 635945.4033144	best: 635945.4033144 (6)	total: 19.5ms	remaining: 9.75s
7:	learn: 363462.2667808	test: 634586.9232946	best: 634586.9232946 (7)	total: 21.9ms	remaining: 9.55s
8:	learn: 359112.0439574	test: 632828.1297363	best: 632828.1297363 (8)	total: 24.3ms	remaining: 9.42s
9:	learn: 354748.1003550	test: 631006.3067663	best: 631006.3067663 (9)	total: 26.6ms

In [5]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(clf, f)

In [6]:
clf.save_model("model")

In [25]:
sr = 0
ind = 0
for a, b in zip(clf.predict(X_test), y_test):
    #print(a, b)
    #print(abs((b-a)/b * 100))
    if b != 0:
      sr+=abs((b-a)/b * 100)
      ind+=1
sr/=ind
sr=100-sr
print(sr) 

[82.78423758599776]


In [26]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

-3465.1763079803195
-3311.588397848041
